# End of game measures
This notebook takes the anonymized data and computes population-level measures for each game.

In [105]:
%pylab inline
import json
import numpy as np
import pandas as pd
import glob
import itertools
from sklearn.decomposition import PCA
from scipy import stats

from helpers import shuffle

Populating the interactive namespace from numpy and matplotlib


/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
output_dir = "../results-anonymized/pilot/"
files = glob.glob(output_dir+'block_*_pilot.json')
files

['../results-anonymized/pilot/block_20200505_pilot.json',
 '../results-anonymized/pilot/block_20200507_pilot.json',
 '../results-anonymized/pilot/block_20200624_pilot.json',
 '../results-anonymized/pilot/block_20200626_pilot.json',
 '../results-anonymized/pilot/block_20200506_pilot.json']

In [3]:
blocks = []
for file in files:
    with open(file) as f:
        blocks.append(json.load(f))

In [76]:
# Enumerate clues to be used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

# Enumerate end-of-game survey questions to be used in polarization analysis
assessments = ['appearance_1', 'appearance_2', 
               'clothing_1', 'clothing_2',
               'suspect_1', 'suspect_2', 'suspect_3',
               'tool_1', 'tool_2', 
               'vehicle_1', 'vehicle_2']    
    
def compute_single_point_measures(game):
    """ 
    Compute the game-level measures 
    
    "Games" in this experiment contain both a treatment and control condition
    and these must be properly separated from one another.
    
    """
    # Form end-of-game survey responses into a dataframe
    collector = {}
    for p, k in game['players'].items():
        try:
            collector[k['data.position']] = k['data.caseMade']
        except:
            print('%s did not complete the post-game survey' %k['data.position'])
    responses = pd.DataFrame(collector).T.sort_index()

    # Form final notebook states into a dataframe
    final_adoptions = pd.DataFrame(data=0, index=responses.index, columns=t_spokes+c_spokes)
    for p, k in game['players'].items():
        for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
            final_adoptions.loc[k['data.position'], clue_id] = 1


    # Determine the number of datapoints to be used in polarization analysis
    # if there are missing responses, need to compare equal sized datasets
    t_responses = [pos for pos in responses.index if pos.startswith('t')]
    c_responses = [pos for pos in responses.index if pos.startswith('c')] 
    # use whichever condition has fewer responses to set the sample size
    n_used = min(len(t_responses), len(c_responses))


    def process_subset(subset, spokes):
        """ compute a result on the selected subset of the data """
        sub_res = {}
        
        # select the subset of the survey responses that will be used in the subset analysis
        sub_survey = responses.loc[subset, assessments]
        
        # survey PC1 
        pca = PCA(n_components=1)
        pca.fit(sub_survey)  
        sub_res['survey PC1'] = pca.explained_variance_ratio_

        # survey similarity percentiles
        survey_corrs = sub_survey.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['survey 5% similarity'], sub_res['survey 95% similarity'] = np.percentile(
            survey_corrs, [5, 95])
        
        # select the subset of the behavioral responses that will be used in the subset analysis
        sub_adopt = final_adoptions.loc[subset, spokes]
        
        # final-state PC1
        pca = PCA(n_components=1)
        pca.fit(sub_adopt)  
        sub_res['spoke PC1'] = pca.explained_variance_ratio_
        
        # final state similarity percentiles
        spoke_corrs = sub_adopt.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['spoke 5% similarity'], sub_res['spoke 95% similarity'] = np.percentile(
            spoke_corrs, [5, 95])
        
        # compute the expected values for the given level of adoption
        # by shuffling the clues between individuals 
        # (preserving the number of clues each individual holds, 
        # and the number of individuals holding each clue)
        # do this a number of times and average the result
        e95 = []
        e5 = []
        ePC1 = []
        for _ in range(100):
            shuffle_adopt = pd.DataFrame(index=sub_adopt.index,
                                         columns=sub_adopt.columns,
                                         data=shuffle(sub_adopt.values, n=500))

            n_agents = len(shuffle_adopt.index)
            corrs = shuffle_adopt.astype(float).T.corr().mask(np.tri(n_agents, n_agents, 0, dtype='bool')).stack()
            e95.append(np.percentile(corrs, 95))
            e5.append(np.percentile(corrs, 5))

            pca = PCA(n_components=1)
            pca.fit(shuffle_adopt)
            ePC1.append(pca.explained_variance_ratio_[0])
        
        # compute the net effect of (interdependent or independent) diffusion 
        # over chance distribution of the same clues
        sub_res['net spoke PC1'] = sub_res['spoke PC1'] - np.mean(ePC1)
        sub_res['net spoke 95% similarity'] = sub_res['spoke 95% similarity'] - np.mean(e95)
        sub_res['net spoke 5% similarity'] = sub_res['spoke 5% similarity'] - np.mean(e5)
        
        return sub_res
        
        
    # For each subset of size 'n_used', compute a result. 
    # In most cases there are no missing responses, so just compute on the complete set
    t_collector = []
    for subset in itertools.combinations(t_responses, r=n_used):
        t_collector.append(process_subset(subset, t_spokes))

    # The recorded result is the average over all subsets
    if len(t_collector) > 1:
        print('Averaging over %i combinations for treatment case'%len(t_collector))
    t_result = pd.DataFrame(t_collector).mean()

    # Compute average for confidence and consensus measures on all submissions
    t_result['confidence'] = responses.loc[t_responses, 'confidence'].mean()
    t_result['consensus'] = responses.loc[t_responses, 'consensus'].mean()    

    
    # Perform the same analysis as above for the control condition
    c_collector = []
    for subset in itertools.combinations(c_responses, r=n_used):
        c_collector.append(process_subset(subset, c_spokes))

    if len(c_collector) > 1:
        print('Averaging over %i combinations for control case'%len(c_collector))
    c_result = pd.DataFrame(c_collector).mean()

    c_result['confidence'] = responses.loc[c_responses, 'confidence'].mean()
    c_result['consensus'] = responses.loc[c_responses, 'consensus'].mean()     

    
    #pd.merge(t_result, c)result, suffixes=(' (inter)', ' (indep)'))
    result = pd.concat([t_result, c_result], keys=['inter', 'indep'])
    #result['game_id']=game['createdAt'].split('_')[0].replace('-','_').replace(':','_').replace('.','_')
    return result

def compute_block(block):
    results_collector = []
    network_collector = []
    for name, game in block.items():
        network_collector.append('caveman' if 'caveman' in name else 'dodec')
        results_collector.append(compute_single_point_measures(game))
        
    result = pd.concat(results_collector, keys=network_collector)
    return result

In [78]:
measurements = pd.concat([compute_block(block) for block in blocks], axis=1)
measurements

In [106]:
def effect_size(measure1, measure2):
    return (measure1 - measure2).mean()

def p_val(measure1, measure2):
    return stats.ttest_rel(measure1, measure2)[1]

def make_table(measurements, func):
    rows = measurements.index.levels[2]
    cols = np.unique(measurements.index.droplevel(2))
    res = pd.DataFrame(index=rows, columns=cols)
    for row in rows:
        for col in cols:
            res.at[row, col] = func(measurements.loc[col+tuple([row])], measurements.loc[('dodec', 'indep')+tuple([row])])
    return res

In [104]:
make_table(measurements, effect_size)

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,2.4,6.88333,0,4.39
consensus,3.15,8.71667,0,3.95
net spoke 5% similarity,-0.0575352,-0.00509524,0,-0.0296869
net spoke 95% similarity,0.129209,0.228675,0,-0.0193147
net spoke PC1,0.132892,0.178375,0,0.0183607
spoke 5% similarity,-0.14851,-0.159686,0,-0.0679407
spoke 95% similarity,0.106442,0.162549,0,-0.0519284
spoke PC1,0.0675537,0.0983758,0,-0.030231
survey 5% similarity,-0.198728,-0.142882,0,-0.0734243
survey 95% similarity,0.0466303,0.105545,0,-0.0270628


In [109]:
make_table(measurements.dropna(axis=1), p_val)

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0.736415,0.114478,NaN,0.229662
consensus,0.631243,0.0193449,NaN,0.575628
net spoke 5% similarity,0.0911892,0.823853,NaN,0.290228
net spoke 95% similarity,0.0280792,0.0636998,NaN,0.174629
net spoke PC1,0.00891613,0.0493536,NaN,0.769419
spoke 5% similarity,0.263095,0.191935,NaN,0.870416
spoke 95% similarity,0.251753,0.0240253,NaN,0.3424
spoke PC1,0.105343,0.138205,NaN,0.143659
survey 5% similarity,0.0242238,0.237789,NaN,0.846876
survey 95% similarity,0.536669,0.413217,NaN,0.69924
